In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [87]:
class ANN:
    def __init__(self, input_size, hidden_size, output_size, lr):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lr = lr

        #initialize the weights
        self.Wij = np.random.uniform(-0.5, 0.5, (self.input_size, self.hidden_size))
        self.Vjk = np.random.uniform(-0.5, 0.5, (self.hidden_size, self.output_size))

        #initialize the bias
        self.bias_j = np.random.uniform(-0.5, 0.5, (1, self.hidden_size))
        self.bias_k = np.random.uniform(-0.5, 0.5, (1, self.output_size))

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))
    def sigmoid_derivate(self, x):
        return x*(1-x)

    def forward_propagate(self,x):
        self.hidden_input = np.dot(x, self.Wij) + self.bias_j
        self.hidden_output = self.sigmoid(self.hidden_input)

        self.out_input = np.dot(self.hidden_output, self.Vjk) + self.bias_k
        self.out_output = self.sigmoid(self.out_input)

        return self.out_output

    def back_propagate(self, x, y, output):
        error = y - output
        delta_out = error * self.sigmoid_derivate(output)

        hidden_error = delta_out.dot(self.Vjk.T)
        hidden_delta = hidden_error * self.sigmoid_derivate(self.hidden_output)

        self.Vjk += self.hidden_output.T.dot(delta_out)*self.lr
        self.bias_k += np.sum(delta_out)*self.lr

        self.Wij += x.T.dot(hidden_delta)*self.lr
        self.bias_j += np.sum(hidden_delta)*self.lr

    def train(self, x, y, epochs):
        for epoch in range(epochs):
            out = self.forward_propagate(x)
            self.back_propagate(x, y, out)
            if epoch%1000 == 0:
                print(f"Epochs {epoch}: Error {np.mean(np.abs(y-out))}")

In [88]:
df = pd.read_csv("Iris.csv")
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [89]:
df['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [90]:
df['Species'] = df['Species'].replace({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})

In [91]:
arr1 = np.array([df["SepalLengthCm"], df["SepalWidthCm"], df["PetalLengthCm"], df["PetalWidthCm"]]).T
arr2 = np.array([df["Species"]]).T

In [92]:
x_train, x_test, y_train, y_test = train_test_split(arr1, arr2, test_size=0.3, random_state=10)

In [93]:
encoder = OneHotEncoder(categories='auto')
y_train_hot = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()

In [94]:
input_size = x_train.shape[1]
hidden_size = 8
output_size = y_train_hot.shape[1]
epochs = 10000
lr = 0.1

obj_ANN = ANN(input_size, hidden_size, output_size, lr)
obj_ANN.train(x_train, y_train_hot, epochs)

Epochs 0: Error 0.4973697240116815
Epochs 1000: Error 0.0532675407117733
Epochs 2000: Error 0.03092673023766424
Epochs 3000: Error 0.026969290227828173
Epochs 4000: Error 0.02323732204099778
Epochs 5000: Error 0.020913765732350896
Epochs 6000: Error 0.020190425531805174
Epochs 7000: Error 0.014413233377538399
Epochs 8000: Error 0.01378177247813307
Epochs 9000: Error 0.01358078696218849


In [95]:
for i in range(len(x_test)):
    predict = obj_ANN.forward_propagate(x_test[i])
    predicted_class = np.argmax(predict)
    actual_class = y_test[i]
    print(f"{x_test[i]}\t{predicted_class}\t{actual_class}")

[6.3 2.3 4.4 1.3]	1	[1]
[6.4 2.7 5.3 1.9]	2	[2]
[5.4 3.7 1.5 0.2]	0	[0]
[6.1 3.  4.6 1.4]	1	[1]
[5.  3.3 1.4 0.2]	0	[0]
[5.  2.  3.5 1. ]	1	[1]
[6.3 2.5 4.9 1.5]	2	[1]
[5.8 2.7 4.1 1. ]	1	[1]
[5.1 3.4 1.5 0.2]	0	[0]
[5.7 2.8 4.5 1.3]	1	[1]
[5.6 3.  4.5 1.5]	1	[1]
[5.8 2.7 5.1 1.9]	2	[2]
[5.5 2.3 4.  1.3]	1	[1]
[4.9 3.  1.4 0.2]	0	[0]
[5.1 3.8 1.5 0.3]	0	[0]
[6.8 3.  5.5 2.1]	2	[2]
[6.  3.4 4.5 1.6]	1	[1]
[4.4 3.  1.3 0.2]	0	[0]
[5.1 3.7 1.5 0.4]	0	[0]
[5.  3.2 1.2 0.2]	0	[0]
[7.1 3.  5.9 2.1]	2	[2]
[6.4 2.8 5.6 2.2]	2	[2]
[6.2 2.8 4.8 1.8]	2	[2]
[4.8 3.4 1.9 0.2]	0	[0]
[5.9 3.  4.2 1.5]	1	[1]
[4.7 3.2 1.3 0.2]	0	[0]
[5.7 3.  4.2 1.2]	1	[1]
[5.5 2.6 4.4 1.2]	1	[1]
[6.8 2.8 4.8 1.4]	1	[1]
[7.7 3.8 6.7 2.2]	2	[2]
[6.6 2.9 4.6 1.3]	1	[1]
[6.2 2.9 4.3 1.3]	1	[1]
[7.2 3.  5.8 1.6]	2	[2]
[5.8 2.8 5.1 2.4]	2	[2]
[6.3 2.5 5.  1.9]	2	[2]
[4.6 3.2 1.4 0.2]	0	[0]
[6.7 3.3 5.7 2.1]	2	[2]
[6.9 3.2 5.7 2.3]	2	[2]
[7.7 2.6 6.9 2.3]	2	[2]
[6.9 3.1 5.1 2.3]	2	[2]
[5.  3.4 1.6 0.4]	0	[0]
[5.  3.5 1.6 0.6

In [96]:
count = 0
total = len(x_test)
for i in range(total):
    predicted = obj_ANN.forward_propagate(x_test[i])
    predicted_class = np.argmax(predicted)
    if predicted_class == y_test[i]:
        count += 1
accuracy = (count/total) * 100
print(accuracy)

97.77777777777777
